<a href="https://colab.research.google.com/github/mendiang/IYKRA-Data-Science-Fellowship-Batch-6/blob/main/Python_Practice_IYKRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##CASE 1

In [2]:
import pandas as pd
import sqlite3 as sql

In [16]:
def create_database_transaction(data):
    con = sql.connect(data)
    cursor = con.cursor()
    cursor.execute("""CREATE TABLE IF NOT EXISTS transactions( id UUID,
                                                               insert_time TIMESTAMP,
                                                               tx_amount FLOAT,
                                                               tx_type TEXT,
                                                               status STRING)""")
    con.commit()

def update_transaction(data):
    con = sql.connect(data)
    cursor = con.cursor()
    sqlite_insert_query = """INSERT INTO transactions
    (id, insert_time, tx_amount, tx_type, status)
    VALUES (?, ?, ?, ?, ?)"""

    transaction_list = [(1, '2021-06-29 02:30:31', 10, 'sell', 'open'),
                        (1, '2021-06-29 02:33:00', 10, 'buy', 'closed'),
                        (3, '2021-06-29 02:45:41', 8, 'sell', 'open'),
                        (4, '2021-06-29 03:30:03', 12, 'sell', 'open'),
                        (5, '2021-06-29 03:10:22', 16, 'buy', 'open'),
                        (6, '2021-06-29 04:00:15', 20, 'sell', 'open'),
                        (4, '2021-06-29 06:20:37', 10, 'sell', 'open'),
                        (8, '2021-06-29 07:25:35', 20, 'buy', 'open'),
                        (9, '2021-06-29 09:02:12', 10, 'buy', 'closed'),
                        (8, '2021-06-29 10:51:33', 15, 'sell', 'open')]

    cursor.executemany(sqlite_insert_query, transaction_list)

    # Query for deleting duplicates
    cursor.execute("""DELETE FROM transactions
                      WHERE insert_time NOT IN
                      (SELECT MAX(insert_time)
                      FROM transactions 
                      GROUP BY id)""") 
    
    con.commit()
 
def view_transaction(data):
    update_transaction(data)
    con = sql.connect(data)
    transaction = pd.read_sql_query('SELECT * FROM transactions', con)
    return transaction



In [17]:
create_database_transaction('transaction.db')

In [18]:
view_transaction('transaction.db')

,id,insert_time,tx_amount,tx_type,status
0,1,2021-06-29 02:33:00,10.0,buy,closed
1,3,2021-06-29 02:45:41,8.0,sell,open
2,5,2021-06-29 03:10:22,16.0,buy,open
3,6,2021-06-29 04:00:15,20.0,sell,open
4,4,2021-06-29 06:20:37,10.0,sell,open
5,9,2021-06-29 09:02:12,10.0,buy,closed
6,8,2021-06-29 10:51:33,15.0,sell,open


##CASE 2


In [8]:
con = sql.connect('transaction.db')
cursor = con.cursor()

In [9]:
cursor.execute('''CREATE TABLE IF NOT EXISTS sales
(Date DATE, Sales INT)''')

In [10]:
# insert values into table
sales_values = [('2021-01-01', 100),
                ('2021-01-02', 34),
                ('2021-01-02', 123),
                ('2021-01-02', 134),
                ('2021-01-03', 145),
                ('2021-01-03', 24),
                ('2021-01-04', 541),
                ('2021-01-04', 636),
                ('2021-01-05', 322),
                ('2021-01-06', 242),
                ('2021-01-07', 22),
                ('2021-01-08', 46),]

cursor.executemany('''INSERT INTO sales values (?,?)''', sales_values)

con.commit()

In [20]:
sales_view = pd.read_sql_query(
             'SELECT Date, sum(Sales) as Total_Sales FROM sales GROUP BY Date', con)

In [21]:
sales_view

,Date,Total_Sales
0,2021-01-01,100
1,2021-01-02,291
2,2021-01-03,169
3,2021-01-04,1177
4,2021-01-05,322
5,2021-01-06,242
6,2021-01-07,22
7,2021-01-08,46


##CASE 3


In [24]:
datasets = [
   {
       "user": {
           "id": 1,
           "name": "agus",
           "email": "agus@example.com"
       },
       "txs": [
           {
               "id": 1,
               "tx_date": "2020-01-01",
               "tx_amount": 10,
               "tx_type": "buy"
           },
           {
               "id": 3,
               "tx_date": "2020-02-01",
               "tx_amount": 10,
               "tx_type": "buy"
           },
           {
               "id": 5,
               "tx_date": "2020-03-01",
               "tx_amount": 20,
               "tx_type": "sell"
           }
       ]
   },
   {
       "user": {
           "id": 2,
           "name": "asep",
           "email": "asep@example.com"
       },
       "txs": [
           {
               "id": 2,
               "tx_date": "2020-01-01",
               "tx_amount": 10,
               "tx_type": "sell"
           },
            {
               "id": 4,
               "tx_date": "2020-02-01",
               "tx_amount": 10,
               "tx_type": "sell"
           },
           {
               "id": 6,
               "tx_date": "2020-03-01",
               "tx_amount": 20,
               "tx_type": "buy"
           }
       ]
   }
] 

In [25]:
datasets

[{'txs': [{'id': 1,
    'tx_amount': 10,
    'tx_date': '2020-01-01',
    'tx_type': 'buy'},
   {'id': 3, 'tx_amount': 10, 'tx_date': '2020-02-01', 'tx_type': 'buy'},
   {'id': 5, 'tx_amount': 20, 'tx_date': '2020-03-01', 'tx_type': 'sell'}],
  'user': {'email': 'agus@example.com', 'id': 1, 'name': 'agus'}},
 {'txs': [{'id': 2,
    'tx_amount': 10,
    'tx_date': '2020-01-01',
    'tx_type': 'sell'},
   {'id': 4, 'tx_amount': 10, 'tx_date': '2020-02-01', 'tx_type': 'sell'},
   {'id': 6, 'tx_amount': 20, 'tx_date': '2020-03-01', 'tx_type': 'buy'}],
  'user': {'email': 'asep@example.com', 'id': 2, 'name': 'asep'}}]

In [28]:
transaction_list = pd.json_normalize(datasets, 'txs',[['user','id'],['user','name'],['user','email']],errors='ignore')

In [29]:
transaction_list

,id,tx_date,tx_amount,tx_type,user.id,user.name,user.email
0,1,2020-01-01,10,buy,1,agus,agus@example.com
1,3,2020-02-01,10,buy,1,agus,agus@example.com
2,5,2020-03-01,20,sell,1,agus,agus@example.com
3,2,2020-01-01,10,sell,2,asep,asep@example.com
4,4,2020-02-01,10,sell,2,asep,asep@example.com
5,6,2020-03-01,20,buy,2,asep,asep@example.com


In [30]:
csv_1 = transaction_list[['user.id','id']]

In [31]:
csv_1

,user.id,id
0,1,1
1,1,3
2,1,5
3,2,2
4,2,4
5,2,6


In [32]:
csv_1_file = csv_1.to_csv('csv_1')

In [33]:
csv_2 = transaction_list[['user.id','user.name', 'user.email']]

In [34]:
csv_2

,user.id,user.name,user.email
0,1,agus,agus@example.com
1,1,agus,agus@example.com
2,1,agus,agus@example.com
3,2,asep,asep@example.com
4,2,asep,asep@example.com
5,2,asep,asep@example.com


In [35]:
csv_2_file = csv_2.to_csv('csv_2')

In [36]:
csv_3 = transaction_list[['id', 'tx_date', 'tx_amount', 'tx_type']]

In [37]:
csv_3

,id,tx_date,tx_amount,tx_type
0,1,2020-01-01,10,buy
1,3,2020-02-01,10,buy
2,5,2020-03-01,20,sell
3,2,2020-01-01,10,sell
4,4,2020-02-01,10,sell
5,6,2020-03-01,20,buy


In [38]:
csv_3_file = csv_3.to_csv('csv_3')